# Ejercicio 3: 

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de ADN que leas de un fichero en formato FASTA. El programa debe hacer la búsqueda en la base de datos nr y filtrar los resultados por el organismo que elijas. El programa debe mostrar por pantalla el número de resultados obtenidos y el E-value medio de los mismos. Hágalo de forma online y local.

### 1.1. Instalación de BLAST

1. Se descarga la versión para el equipo de BLAST desde el siguiente enlace: [BLAST](https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/)
2. Se ejecuta, en nuestro caso, el *.exe* descargado y se instala en el equipo.
3. Se configura el *path* de BLAST como variable de entorno del sistema.
4. Se verifica la instalación con el comando `blastn --version`.

### 1.2. Secuencia de ADN por teclado

In [1]:
from processing.data_loader import DataLoaderFactory

In [2]:
sequence = DataLoaderFactory.get_loader('api').load(('NM_000518.5',), 'nucleotide')

File NM_000518_5.fasta created


Así, se ha obtenido la siguiente secuencias de ADN para realizar la búsqueda BLAST:

In [3]:
sequence

[SeqRecord(seq=Seq('ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGG...CAA'), id='NM_000518.5', name='NM_000518.5', description='NM_000518.5 Homo sapiens hemoglobin subunit beta (HBB), mRNA', dbxrefs=[])]

No obstante, nos interesa quedarnos únicamente con la secuencia en sí:

In [4]:
sequence = sequence[0].seq

### 1.3. Implementación online

In [5]:
from Bio.Blast import NCBIWWW, NCBIXML

In [32]:
def search_online_sequence(sequence, organism_filter):
    result = NCBIWWW.qblast("blastn", "nr", sequence)
    blast_records = NCBIXML.read(result)

    print(f"Resultados BLAST para la secuencia sin filtrar por organismo:")
    print(f"Número de alineamientos: {len(blast_records.alignments)}")

    best_alignment = blast_records.alignments[0]
    best_hsp = best_alignment.hsps[0]
    e_value = best_hsp.expect
    description = best_alignment.title

    print(f"E-value del mejor resultado: {e_value}")
    print(f"Descripción de la secuencia más similar: {description}")

    if not blast_records.alignments:
        print("No se encontraron resultados.")
        return
    
    filtered_alignments = []
    for alignment in blast_records.alignments:
        if organism_filter.lower() in alignment.title.lower():
            filtered_alignments.append(alignment)

    if not filtered_alignments:
        print(f"No se encontraron resultados para el organismo '{organism_filter}'.")
        return
    
    num_results = len(filtered_alignments)
    
    best_alignment = filtered_alignments[0]
    best_hsp = best_alignment.hsps[0]
    e_value = best_hsp.expect
    description = best_alignment.title
    
    print("\nResultados BLAST filtrados por organismo:")
    print(f"Número de resultados: {num_results}")
    print(f"E-value del mejor resultado: {e_value}")
    print(f"Descripción de la secuencia más similar: {description}")

In [33]:
search_online_sequence(sequence, 'Homo sapiens')

Resultados BLAST para la secuencia sin filtrar por organismo:
Número de alineamientos: 50
E-value del mejor resultado: 0.0
Descripción de la secuencia más similar: gi|1401724401|ref|NM_000518.5| Homo sapiens hemoglobin subunit beta (HBB), mRNA

Resultados BLAST filtrados por organismo:
Número de resultados: 18
E-value del mejor resultado: 0.0
Descripción de la secuencia más similar: gi|1401724401|ref|NM_000518.5| Homo sapiens hemoglobin subunit beta (HBB), mRNA


En efecto, se ha encontrado la secuencia que se buscaba. Probaremos ahora a modificar la secuencia, añadiendo mutaciones aleatorias, con el fin de comprobar si el programa es capaz de encontrar la secuencia original.

In [34]:
import random

In [35]:
def introduce_mutation(sequence, num_mutations=1):
    bases = ['A', 'T', 'C', 'G']
    sequence_list = list(sequence)

    for _ in range(num_mutations):
        mutation_idx = random.choice(range(len(sequence)))
        current_base = sequence_list[mutation_idx]
        possible_bases = [base for base in bases if base != current_base]
        new_base = random.choice(possible_bases)
        sequence_list[mutation_idx] = new_base
    
    mutated_sequence = ''.join(sequence_list)
    return mutated_sequence

In [36]:
mutated_sequence = introduce_mutation(sequence, num_mutations=15)
print(sequence)
print(mutated_sequence)

ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGAGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGCTGCTGGTGGTCTACCCTTGGACCCAGAGGTTCTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTGCACGTGGATCCTGAGAACTTCAGGCTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAAGAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAAGCTCGCTTTCTTGCTGTCCAATTTCTATTAAAGGTTCCTTTGTTCCCTAAGTCCAACTACTAAACTGGGGGATATTATGAAGGGCCTTGAGCATCTGGATTCTGCCTAATAAAAAACATTTATTTTCATTGCAA
AAATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGAGGAGAAGTTTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGTTGAGGCCGGGGGCAGTCTGCTGGTGGTCTACCCTTGGACCCAGAGGTTCTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGAGCACCTTTGTCACACTGAGTGAGCTGCACTGTGACAGGCTGCACGTGGATCCTGAGAACTTCAGGCTCCTG

In [37]:
search_online_sequence(mutated_sequence, 'Homo sapiens')

Resultados BLAST para la secuencia sin filtrar por organismo:
Número de alineamientos: 50
E-value del mejor resultado: 0.0
Descripción de la secuencia más similar: gi|1401724401|ref|NM_000518.5| Homo sapiens hemoglobin subunit beta (HBB), mRNA

Resultados BLAST filtrados por organismo:
Número de resultados: 17
E-value del mejor resultado: 0.0
Descripción de la secuencia más similar: gi|1401724401|ref|NM_000518.5| Homo sapiens hemoglobin subunit beta (HBB), mRNA


Vemos como seguimos obteniendo el mismo resultado. Esto es evidente, puesto que tan solo se ha alterado un 2.3% de la secuencia original.

In [39]:
15/len(sequence)

0.02388535031847134

 Ahora bien, ¿y si filtrásemos para un organismo distinto al humano, del que se ha extarído la secuencia de ADN? Veamos, para ello, cuales son los organismos que se han obtenido, y para qué secuencias se han obtenido.

In [6]:
import pandas as pd

In [11]:
def search_online_sequences(sequence):
    result = NCBIWWW.qblast("blastn", "nr", sequence)
    blast_records = NCBIXML.read(result)

    data = []

    if not blast_records.alignments:
        print("No se encontraron resultados.")
        return pd.DataFrame()
    
    for alignment in blast_records.alignments:
        organism = alignment.title
        sequence_data = alignment.hsps[0].align_length
        e_value = alignment.hsps[0].expect
        
        data.append({
            "Organismo": organism,
            "Secuencia": sequence_data,
            "E-value": e_value
        })
    
    if not data:
        print(f"No se encontraron resultados.")
        return pd.DataFrame()
    
    df = pd.DataFrame(data)
    return df

In [12]:
results = search_online_sequences(sequence)

In [21]:
hemoglobin = 0
total = 0

for i in range(len(results)):
    print(results['Organismo'][i])
    if 'hemoglobin' in results['Organismo'][i].lower():
        hemoglobin += 1
    total += 1

gi|1401724401|ref|NM_000518.5| Homo sapiens hemoglobin subunit beta (HBB), mRNA
gi|2468480781|ref|XM_508242.5| PREDICTED: Pan troglodytes hemoglobin subunit beta (HBB), mRNA
gi|2694496772|ref|XM_003819029.5| PREDICTED: Pan paniscus hemoglobin subunit beta (LOC100976465), mRNA
gi|13937928|gb|BC007075.1| Homo sapiens hemoglobin, beta, mRNA (cDNA clone MGC:14540 IMAGE:4292125), complete cds
gi|29436|emb|V00497.1| Human messenger RNA for beta-globin
gi|40886940|gb|AY509193.1| Homo sapiens hemoglobin beta mRNA, complete cds
gi|2695096866|ref|XM_019036164.3| PREDICTED: Gorilla gorilla gorilla hemoglobin subunit beta (LOC101126932), mRNA
gi|2694676380|ref|XM_002822127.6| PREDICTED: Pongo abelii hemoglobin subunit beta (HBB), mRNA
gi|2695876965|ref|XM_054440830.2| PREDICTED: Pongo pygmaeus hemoglobin subunit beta (HBB), mRNA
gi|2491688240|ref|XM_055282707.1| PREDICTED: Symphalangus syndactylus hemoglobin subunit beta (HBB), mRNA
gi|1743165722|ref|XM_004090649.3| PREDICTED: Nomascus leucogenys 

In [23]:
per = hemoglobin/total
print(f"Un {per*100}% de los resultados contienen la palabra 'hemoglobin'.")

Un 86.0% de los resultados contienen la palabra 'hemoglobin'.


### 1.4. Implementación local

Una vez instalado blast, se procede a descargar la base de datos *nr* que se utilizará para la búsqueda. Para ello, se ejecuta el siguiente comando:


In [22]:
!perl update_blastdb.pl --decompress nr

"perl" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
